# List question

Il modello per le liste si basa sullo stesso modello utilizzato per le factoid provvediamo a importare il modello corretto e facciamo un test sul funzionamento

In [1]:
from transformers import BertTokenizer, BertModel, BertForQuestionAnswering
import torch
tokenizer = BertTokenizer.from_pretrained("./embedding_models/biobert_factoid")
model = BertForQuestionAnswering.from_pretrained("./embedding_models/biobert_factoid")

In [22]:
question, text = "List signaling molecules (ligands) that interact with the receptor EGFR?", "the epidermal growth factor receptor (EGFR) ligands, such as epidermal growth factor (EGF) and amphiregulin (AREG)"
encoding = tokenizer.encode_plus(question, text)
input_ids, token_type_ids = encoding["input_ids"], encoding["token_type_ids"]
all_tokens = tokenizer.convert_ids_to_tokens(input_ids)
print(all_tokens)
print(input_ids)
print(token_type_ids)

['[CLS]', 'list', 'signaling', 'molecules', '(', 'l', '##igan', '##ds', ')', 'that', 'interact', 'with', 'the', 'receptor', 'e', '##g', '##f', '##r', '?', '[SEP]', 'the', 'e', '##pid', '##er', '##mal', 'growth', 'factor', 'receptor', '(', 'e', '##g', '##f', '##r', ')', 'l', '##igan', '##ds', ',', 'such', 'as', 'e', '##pid', '##er', '##mal', 'growth', 'factor', '(', 'e', '##g', '##f', ')', 'and', 'am', '##phi', '##re', '##gu', '##lin', '(', 'are', '##g', ')', '[SEP]']
[101, 2190, 16085, 10799, 113, 181, 10888, 3680, 114, 1115, 12254, 1114, 1103, 10814, 174, 1403, 2087, 1197, 136, 102, 1103, 174, 25786, 1200, 7435, 3213, 5318, 10814, 113, 174, 1403, 2087, 1197, 114, 181, 10888, 3680, 117, 1216, 1112, 174, 25786, 1200, 7435, 3213, 5318, 113, 174, 1403, 2087, 114, 1105, 1821, 27008, 1874, 13830, 2836, 113, 1132, 1403, 114, 102]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

In [23]:
start_scores, end_scores = model(torch.tensor([input_ids]), token_type_ids=torch.tensor([token_type_ids]))
print(start_scores)
print(end_scores)
print(torch.argmax(start_scores))
print(torch.argmax(end_scores))

tensor([[-3.5126, -7.0865, -6.9067, -7.7472, -7.6168, -6.4265, -7.8138, -8.5346,
         -8.5282, -7.4421, -7.2022, -7.1606, -7.4848, -8.1219, -7.4963, -7.8732,
         -8.4675, -8.8533, -7.7126, -7.8307, -2.8615, -3.0179, -6.7289, -6.7677,
         -6.4157, -6.0162, -6.7201, -4.8849, -4.4312, -5.3506, -7.3976, -7.3558,
         -6.2990, -4.9410, -2.0951, -5.2530, -4.9682, -5.6626, -3.1414, -6.0405,
         -0.9388, -7.2085, -7.6842, -6.7369, -5.8507, -6.5148, -5.1844, -4.1726,
         -7.7461, -7.0326, -7.0507, -7.4014, -1.8171, -7.4988, -7.5553, -7.1843,
         -5.8848, -5.8190, -6.7066, -7.4752, -6.2458, -7.8307]],
       grad_fn=<SqueezeBackward1>)
tensor([[-2.4085, -8.4290, -8.2037, -6.7629, -8.2680, -8.3713, -8.0804, -6.4559,
         -7.4831, -8.3483, -8.1218, -8.0534, -8.5519, -7.9593, -8.7100, -8.3001,
         -7.9757, -6.8480, -7.3486, -2.5235, -5.5465, -5.8049, -7.4287, -7.5961,
         -7.3014, -6.4549, -4.0458, -2.8896, -5.6771, -8.3694, -8.4386, -8.0441,
         

In [24]:

answer = ' '.join(all_tokens[torch.argmax(start_scores) : torch.argmax(end_scores)+1])
print(answer)

In [25]:
# Start with the first token.
answer_start=torch.argmax(start_scores)
answer_end=torch.argmax(end_scores)
answer = all_tokens[answer_start]

# Select the remaining answer tokens and join them with whitespace.
for i in range(answer_start + 1, answer_end + 1):
    
    # If it's a subword token, then recombine it with the previous token.
    if all_tokens[i][0:2] == '##':
        answer += all_tokens[i][2:]
    
    # Otherwise, add a space then the token.
    else:
        answer += ' ' + all_tokens[i]

print('Answer: "' + answer + '"')

Answer: "e"


In [28]:
start_scores, end_scores = model(torch.tensor([input_ids]), token_type_ids=torch.tensor([token_type_ids]))
print(start_scores)
print(end_scores)
values_start, indices_start=torch.topk(start_scores,3)
values_end, indices_end=torch.topk(end_scores,3)
print(indices_start)
print(indices_end)

tensor([[-3.5126, -7.0865, -6.9067, -7.7472, -7.6168, -6.4265, -7.8138, -8.5346,
         -8.5282, -7.4421, -7.2022, -7.1606, -7.4848, -8.1219, -7.4963, -7.8732,
         -8.4675, -8.8533, -7.7126, -7.8307, -2.8615, -3.0179, -6.7289, -6.7677,
         -6.4157, -6.0162, -6.7201, -4.8849, -4.4312, -5.3506, -7.3976, -7.3558,
         -6.2990, -4.9410, -2.0951, -5.2530, -4.9682, -5.6626, -3.1414, -6.0405,
         -0.9388, -7.2085, -7.6842, -6.7369, -5.8507, -6.5148, -5.1844, -4.1726,
         -7.7461, -7.0326, -7.0507, -7.4014, -1.8171, -7.4988, -7.5553, -7.1843,
         -5.8848, -5.8190, -6.7066, -7.4752, -6.2458, -7.8307]],
       grad_fn=<SqueezeBackward1>)
tensor([[-2.4085, -8.4290, -8.2037, -6.7629, -8.2680, -8.3713, -8.0804, -6.4559,
         -7.4831, -8.3483, -8.1218, -8.0534, -8.5519, -7.9593, -8.7100, -8.3001,
         -7.9757, -6.8480, -7.3486, -2.5235, -5.5465, -5.8049, -7.4287, -7.5961,
         -7.3014, -6.4549, -4.0458, -2.8896, -5.6771, -8.3694, -8.4386, -8.0441,
         

In [30]:
answer = ' '.join(all_tokens[34 : 56])
print(answer)

l ##igan ##ds , such as e ##pid ##er ##mal growth factor ( e ##g ##f ) and am ##phi ##re ##gu


In [31]:
from bert import Ner


ModuleNotFoundError: No module named 'bert'